# Reviewing Data of RPG Steam Games using Python and Pandas
Data Source: https://steamspy.com/ 19 July 2021

Created Jupyter Notebook in Google Colabotary

In [74]:
import numpy as np
import pandas as pd


Uploaded data to Google drive and linked to notebook.

In [ ]:
df = pd.read_csv('drive/MyDrive/steamrpg.csv')

In [ ]:
df.head()

,#,Game,Release date,Price,Score rank(Userscore / Metascore),Owners,Playtime (Median),Developer(s),Publisher(s)
0,120,Terraria,"May 16, 2011",$9.99,N/A (N/A/83%),"20,000,000 .. 50,000,000",06:23 (02:48),Re-Logic,Re-Logic
1,525,Rust,"Feb 8, 2018",$39.99,N/A (N/A/69%),"20,000,000 .. 50,000,000",19:20 (09:08),Facepunch Studios,Facepunch Studios
2,10,The Witcher 3: Wild Hunt,"May 18, 2015",$39.99,N/A (N/A/93%),"10,000,000 .. 20,000,000",16:56 (02:10),CD PROJEKT RED,CD PROJEKT RED
3,33,Borderlands 2,"Sep 17, 2012",$19.99,N/A (N/A/89%),"10,000,000 .. 20,000,000",02:08 (02:52),"Gearbox Software, Aspyr (Mac), Aspyr (Linux)","2K, Aspyr (Mac), Aspyr (Linux)"
4,34,Stardew Valley,"Feb 26, 2016",$14.99,N/A (N/A/89%),"10,000,000 .. 20,000,000",08:41 (05:35),ConcernedApe,ConcernedApe


Rename first column to AppID

In [60]:
df = df.rename(columns={'#': 'AppID'})

Add columns for min and max approximate number of owners

In [70]:
df['Owners'].unique()

array(['20,000,000\xa0..\xa050,000,000', '10,000,000\xa0..\xa020,000,000',
       '5,000,000\xa0..\xa010,000,000', '2,000,000\xa0..\xa05,000,000',
       '1,000,000\xa0..\xa02,000,000', '500,000\xa0..\xa01,000,000',
       '200,000\xa0..\xa0500,000', '100,000\xa0..\xa0200,000',
       '50,000\xa0..\xa0100,000', '20,000\xa0..\xa050,000',
       '0\xa0..\xa020,000'], dtype=object)

In [61]:
df[['Owners(Min)','Owners(Max)']] = df['Owners'].str.split('\xa0..\xa0',expand=True)

In [62]:
df['Owners(Max)']=df['Owners(Max)'].str.replace(',','')

In [63]:
df['Owners(Max)'] = df['Owners(Max)'].astype(int)

In [64]:
df.head()

,AppID,Game,Release date,Price,Score rank(Userscore / Metascore),Owners,Playtime (Median),Developer(s),Publisher(s),Owners(Min),Owners(Max)
0,120,Terraria,"May 16, 2011",$9.99,N/A (N/A/83%),"20,000,000 .. 50,000,000",06:23 (02:48),Re-Logic,Re-Logic,"20,000,000",50000000
1,525,Rust,"Feb 8, 2018",$39.99,N/A (N/A/69%),"20,000,000 .. 50,000,000",19:20 (09:08),Facepunch Studios,Facepunch Studios,"20,000,000",50000000
2,10,The Witcher 3: Wild Hunt,"May 18, 2015",$39.99,N/A (N/A/93%),"10,000,000 .. 20,000,000",16:56 (02:10),CD PROJEKT RED,CD PROJEKT RED,"10,000,000",20000000
3,33,Borderlands 2,"Sep 17, 2012",$19.99,N/A (N/A/89%),"10,000,000 .. 20,000,000",02:08 (02:52),"Gearbox Software, Aspyr (Mac), Aspyr (Linux)","2K, Aspyr (Mac), Aspyr (Linux)","10,000,000",20000000
4,34,Stardew Valley,"Feb 26, 2016",$14.99,N/A (N/A/89%),"10,000,000 .. 20,000,000",08:41 (05:35),ConcernedApe,ConcernedApe,"10,000,000",20000000


Checking for nulls

In [65]:
df.isnull().sum()

AppID                                  0
Game                                   1
Release date                           0
Price                                355
Score rank(Userscore / Metascore)      0
Owners                                 0
Playtime (Median)                      0
Developer(s)                           3
Publisher(s)                          10
Owners(Min)                            0
Owners(Max)                            0
dtype: int64

In [66]:
df[df['Game'].isnull()]

,AppID,Game,Release date,Price,Score rank(Userscore / Metascore),Owners,Playtime (Median),Developer(s),Publisher(s),Owners(Min),Owners(Max)
2436,5977,NaN,"Jul 19, 2018",$2.99,N/A (N/A),"20,000 .. 50,000",00:00 (00:00),Nuclear Tales,NaN,"20,000",50000


Checking back the website, the missing game is called "The Box." Correcting entry.

In [68]:
df.loc[2436,'Game'] = 'The Box'

In [69]:
df.loc[2436]

AppID                                            5977
Game                                          The Box
Release date                             Jul 19, 2018
Price                                           $2.99
Score rank(Userscore / Metascore)           N/A (N/A)
Owners                               20,000 .. 50,000
Playtime (Median)                       00:00 (00:00)
Developer(s)                            Nuclear Tales
Publisher(s)                                      NaN
Owners(Min)                                    20,000
Owners(Max)                                     50000
Name: 2436, dtype: object

Checking for duplicate games

In [71]:
df.duplicated(subset='Game').sum()

0

Looking at RPG popularity based on approximate number of owners

In [72]:
df['Owners(Max)'].value_counts()

20000       5059
50000       1048
100000       602
500000       425
200000       409
1000000      208
2000000      111
5000000       65
10000000      16
20000000      11
50000000       2
Name: Owners(Max), dtype: int64

In [84]:
df.loc[df['Owners(Max)'] >= 20000000]

,AppID,Game,Release date,Price,Score rank(Userscore / Metascore),Owners,Playtime (Median),Developer(s),Publisher(s),Owners(Min),Owners(Max)
0,120,Terraria,"May 16, 2011",$9.99,N/A (N/A/83%),"20,000,000 .. 50,000,000",06:23 (02:48),Re-Logic,Re-Logic,"20,000,000",50000000
1,525,Rust,"Feb 8, 2018",$39.99,N/A (N/A/69%),"20,000,000 .. 50,000,000",19:20 (09:08),Facepunch Studios,Facepunch Studios,"20,000,000",50000000
2,10,The Witcher 3: Wild Hunt,"May 18, 2015",$39.99,N/A (N/A/93%),"10,000,000 .. 20,000,000",16:56 (02:10),CD PROJEKT RED,CD PROJEKT RED,"10,000,000",20000000
3,33,Borderlands 2,"Sep 17, 2012",$19.99,N/A (N/A/89%),"10,000,000 .. 20,000,000",02:08 (02:52),"Gearbox Software, Aspyr (Mac), Aspyr (Linux)","2K, Aspyr (Mac), Aspyr (Linux)","10,000,000",20000000
4,34,Stardew Valley,"Feb 26, 2016",$14.99,N/A (N/A/89%),"10,000,000 .. 20,000,000",08:41 (05:35),ConcernedApe,ConcernedApe,"10,000,000",20000000
5,64,Path of Exile,"Oct 23, 2013",Free,N/A (N/A/86%),"10,000,000 .. 20,000,000",18:19 (11:21),Grinding Gear Games,Grinding Gear Games,"10,000,000",20000000
6,65,Cyberpunk 2077,"Dec 9, 2020",$59.99,N/A (N/A/86%),"10,000,000 .. 20,000,000",24:40 (11:53),CD PROJEKT RED,CD PROJEKT RED,"10,000,000",20000000
7,232,PAYDAY 2,"Aug 13, 2013",$9.99,N/A (N/A/79%),"10,000,000 .. 20,000,000",20:37 (04:42),OVERKILL - a Starbreeze Studio.,Starbreeze Publishing AB,"10,000,000",20000000
8,647,Warface,"Apr 12, 2012",Free,N/A (N/A/62%),"10,000,000 .. 20,000,000",01:16 (02:06),MY.GAMES,MY.GAMES,"10,000,000",20000000
9,7947,Trove,"Jul 9, 2015",NaN,N/A (N/A),"10,000,000 .. 20,000,000",00:48 (00:56),Trion Worlds,Trion Worlds,"10,000,000",20000000
